<a href="https://colab.research.google.com/github/tanvircr7/meh/blob/master/GNN_Grouping_Research_Papers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ogb
!pip install torch_geometric

# The idea of using those two libraries together is that one can more easily propose an architecture and does not have to worry about the data obtention and the evaluation mechanism.
The PyG is an extension for the Pytorch library which allows us to quickly implement new GNNs architectures using already established layers from research.

The OGB [2] was developed as a way of improving the quality of the research on the area since it provides curated graphs to be used and also a standard way of evaluating the results from a given architecture, making so that the comparison between proposals is fairer.

In [4]:
import os
import torch
import torch.nn.functional as F
from tqdm import tqdm
from torch_geometric.loader import NeighborLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.nn import MessagePassing, SAGEConv
from ogb.nodeproppred import Evaluator, PygNodePropPredDataset

# downloading the dataset from the OGB

In [27]:
target_dataset = 'ogbn-arxiv'
# This will download the ogbn-arxiv to the 'networks' folder
dataset = PygNodePropPredDataset(name=target_dataset, root='networks')

# the number of nodes, the adjacency list, the feature vector for the network, the year for each node, and the target label.

In [28]:
data = dataset[0]
data

Data(num_nodes=169343, edge_index=[2, 1166243], x=[169343, 128], node_year=[169343, 1], y=[169343, 1])

# This network already comes with a split set for training, validation, and testing. This is provided by the OGB as a way of improving reproducibility and the quality of research on this network.

In [29]:
split_idx = dataset.get_idx_split()

train_idx = split_idx['train']
valid_idx = split_idx['valid']
test_idx = split_idx['test']

In [30]:
train_loader = NeighborLoader(data, input_nodes=train_idx,
                              shuffle=True, num_workers=os.cpu_count() - 2,
                              batch_size=1024, num_neighbors=[30] * 2)
total_loader = NeighborLoader(data, input_nodes=None, num_neighbors=[-1],
                               batch_size=4096, shuffle=False,
                               num_workers=os.cpu_count() - 2)

/usr/local/lib/python3.10/dist-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "


# Break Down
We must define the number of in_channels of the network, this will be the number of features in our dataset. The out_channels is going to be the number of classes we are trying to predict. The hidden channels parameter is a value we can define ourselves that represents the number of hidden units.

We can set the number of layers of the network. For each hidden layer, we add a Batch Normalization layer and then we reset the parameters for every layer.

The forward method runs a single iteration of a forward pass. It will get the feature vector and the adjacency list and pass it to the SAGE Layer and then pass the result to the Batch Normalization layer. We also apply a ReLU non-linearity and a dropout layer for regularization purposes.

Finally, the inference method will generate a prediction for every node in the dataset. We will use it for validation purposes.



In [31]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels,
                 hidden_channels, out_channels,
                 n_layers=2):

        super(SAGE, self).__init__()
        self.n_layers = n_layers

        self.layers = torch.nn.ModuleList()
        self.layers_bn = torch.nn.ModuleList()

        if n_layers == 1:
            self.layers.append(SAGEConv(in_channels, out_channels,   normalize=False))

        elif n_layers == 2:
            self.layers.append(SAGEConv(in_channels, hidden_channels, normalize=False))
            self.layers_bn.append(torch.nn.BatchNorm1d(hidden_channels))
            self.layers.append(SAGEConv(hidden_channels, out_channels, normalize=False))

        else:
            self.layers.append(SAGEConv(in_channels, hidden_channels, normalize=False))

            self.layers_bn.append(torch.nn.BatchNorm1d(hidden_channels))
            self.layers.append(SAGEConv(hidden_channels, out_channels, normalize=False))

        for layer in self.layers:
            layer.reset_parameters()

    def forward(self, x, edge_index):
        if len(self.layers) > 1:
            looper = self.layers[:-1]
        else:
            looper = self.layers

        for i, layer in enumerate(looper):
            x = layer(x, edge_index)
            try:
                x = self.layers_bn[i](x)
            except Exception as e:
                abs(1)
            finally:
                x = F.relu(x)
                x = F.dropout(x, p=0.5, training=self.training)

        if len(self.layers) > 1:
            x = self.layers[-1](x, edge_index)
        return F.log_softmax(x, dim=-1), torch.var(x)

    def inference(self, total_loader, device):
        xs = []
        var_ = []
        for batch in total_loader:
            out, var = self.forward(batch.x.to(device), batch.edge_index.to(device))
            out = out[:batch.batch_size]
            xs.append(out.cpu())
            var_.append(var.item())
        out_all = torch.cat(xs, dim=0)

        return out_all, var_



# Parameters for the model

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SAGE(data.x.shape[1], 256, dataset.num_classes, n_layers=2)
model.to(device)
epochs = 100
optimizer = torch.optim.Adam(model.parameters(), lr=0.03)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=7)

# Test Function

In [33]:
def test(model, device):
    evaluator = Evaluator(name=target_dataset)
    model.eval()
    out, var = model.inference(total_loader, device)
    y_true = data.y.cpu()
    y_pred = out.argmax(dim=-1, keepdim=True)
    train_acc = evaluator.eval({
        'y_true': y_true[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    val_acc = evaluator.eval({
        'y_true': y_true[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': y_true[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, val_acc, test_acc, torch.mean(torch.Tensor(var))

# Training Loop

In [34]:
for epoch in range(1, epochs):
    model.train()
    pbar = tqdm(total=train_idx.size(0))
    pbar.set_description(f'Epoch {epoch:02d}')
    total_loss = total_correct = 0

    for batch in train_loader:
        batch_size = batch.batch_size
        optimizer.zero_grad()
        out, _ = model(batch.x.to(device), batch.edge_index.to(device))
        out = out[:batch_size]
        batch_y = batch.y[:batch_size].to(device)
        batch_y = torch.reshape(batch_y, (-1,))
        loss = F.nll_loss(out, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += float(loss)
        total_correct += int(out.argmax(dim=-1).eq(batch_y).sum())
        pbar.update(batch.batch_size)

    pbar.close()
    loss = total_loss / len(train_loader)
    approx_acc = total_correct / train_idx.size(0)
    train_acc, val_acc, test_acc, var = test(model, device)

    print(f'Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}, Var: {var:.4f}')


Epoch 01:   0%|          | 0/90941 [00:00<?, ?it/s]

ImportError: 'NeighborSampler' requires either 'pyg-lib' or 'torch-sparse'